In [1]:
# Extract AMC Data from numina
from concurrent.futures import as_completed
import json
from pprint import pprint
from tqdm import tqdm

from datasets import load_dataset
from rllm.grading.grader import extract_boxed_answer


ds = load_dataset("AI-MO/NuminaMath-CoT")
# Filter for amc_aime problems
olympiad = ds['train'].filter(lambda x: x['source'] == 'olympiads')

/opt/conda/envs/rllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Filter for non-proof problems.
import concurrent.futures
from rllm.utils import call_gemini_llm
from rllm.system_prompts import PROOF_PROMPT

def process_entry(entry):
    output_dict = {}
    # 1) Get the problem text
    problem_text = entry['problem']
    solution_text = entry['solution']
    # 2) Call Gemini LLM
    output_str = call_gemini_llm(f'Problem: {problem_text} \n\n Solution: {solution_text}', system_prompt=PROOF_PROMPT)
    if not output_str:
        print("Gemini not happy.")
        return None
    if '2' in output_str:
        output_dict['problem'] = entry['problem']
        output_dict['solution'] = entry['solution']
        return {
            'problem': entry['problem'],
            'solution': entry['solution'],
        }
    else:
        if '3' in output_str:
            print(output_str, entry['problem'])
    return output_dict

# Suppose `olympiad` is your list of dictionaries
subset = olympiad  
results = []

with concurrent.futures.ProcessPoolExecutor(max_workers=48) as executor:
    # 1) Submit all jobs to the executor
    futures = [executor.submit(process_entry, entry) for entry in subset]

# 2) Process them as they complete, using tqdm for a progress bar
for future in tqdm(as_completed(futures), total=len(futures), desc="Processing entries"):
    # Get the result for each completed future
    result = future.result()
    if result:
        results.append(result)


# Save final list as json
with open("olympiad_processed.json", "w") as f:
    json.dump(results, f, indent=2)

3
 You see a square resembling a chessboard that is divided into 4 parts. Can you arrange these parts in such a way that the new figure has one less cell, i.e., 63 cells?
3
 How to easily and accurately remember the formulas for the sine and cosine of angles:

$$
\frac{\pi}{2} \pm \alpha ; \pi \pm \alpha ; \frac{3 \pi}{2} \pm \alpha, 2 \pi - \alpha ?
$$
3
 In one glass, there was milk, and in another - the same amount of coffee. A spoonful was transferred from the glass of milk to the glass of coffee and mixed. Then, the same spoonful of the mixture was transferred back to the glass with milk. What is there more of now: coffee in the glass with milk or milk in the glass with coffee?
3
 Cut a triangle into two parts, which can be reassembled into a 20-sided polygon.
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhauste

Processing entries: 100%|██████████| 102270/102270 [00:00<00:00, 104646.63it/s]


In [1]:
from rllm.utils import RAG
import json

with open("numina_olympiad.json", 'r', encoding='utf-8') as f:
    olympiad_data = json.load(f)

with open("../raw/train/omni_math.json", 'r', encoding='utf-8') as f:
    omni_data = json.load(f)

rag_searcher = RAG(docs=[d["problem"] for d in omni_data])

/opt/conda/envs/rllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Filter for olympiad problems that are not in the omni dataset
from tqdm import tqdm

filter_olympiad = []
num_problems = 0

counter = 0
# Wrap olympiad_data with tqdm, optionally adding a description and total
for d in tqdm(olympiad_data, desc="Filtering olympiad data", total=len(olympiad_data)):
    search_result = rag_searcher.top_k(d["problem"], k=1)[0]
    score = search_result["score"]
    if score > 0.93:
        num_problems += 1
    else:
        filter_olympiad.append(d)
    counter += 1
    if counter %1000 == 0:
        print(counter)
# Save final list as json
with open("olympiad_filter.json", "w") as f:
    json.dump(filter_olympiad, f, indent=2)
num_problems

Filtering olympiad data:   1%|          | 1016/102125 [00:07<12:49, 131.44it/s]

1000


Filtering olympiad data:   2%|▏         | 2021/102125 [00:15<12:46, 130.56it/s]

2000


Filtering olympiad data:   3%|▎         | 3015/102125 [00:23<12:37, 130.76it/s]

...

101000


Filtering olympiad data: 100%|█████████▉| 102021/102125 [12:52<00:00, 132.25it/s]

102000


Filtering olympiad data: 100%|██████████| 102125/102125 [12:53<00:00, 132.05it/s]


2491

In [5]:
import json
# Process dataset to prpoduce answers
with open("olympiad_filter.json", 'r', encoding='utf-8') as f:
    olympiad = json.load(f)


In [ ]:
# Have gemini add the solutions to the numina dataset.
# Filter for non-proof problems.
import concurrent.futures
from concurrent.futures import as_completed
from rllm.utils import call_gemini_llm
from rllm.system_prompts import SOLUTION_PROMPT

def get_answer(entry):
    # 1) Get the problem text
    problem_text = entry['problem']
    solution_text = entry['solution']
    # 2) Call Gemini LLM
    output_str = call_gemini_llm(f'Problem: {problem_text} \n----\n Solution: {solution_text}', system_prompt=SOLUTION_PROMPT)
    if 'error' in output_str or 'Error' in output_str or 'Solution not found' in output_str:
        print(problem_text)
        print(solution_text)
        return None
    if 'answer' in entry:
        if output_str in entry['answer']:
            return entry
        entry['answer'].append(output_str)
    else:
        entry['answer'] = [output_str]
    return entry

results = []
idx = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=48) as executor:
    # 1) Submit all jobs to the executor
    futures = [executor.submit(get_answer, entry) for entry in olympiad]
    # 2) Process them as they complete, using tqdm for a progress bar
    for future in as_completed(futures):
        # Get the result for each completed future
        result = future.result()
        if result:
            results.append(result)
        if idx%1000 == 0:
            print(idx)
            with open("olympiad_solutions.json", "w") as f:
                json.dump(results, f, indent=2)
        idx += 1

# Save final list as json
with open("olympiad_solutions.json", "w") as f:
    json.dump(results, f, indent=2)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
Retry due to rate limit:  429 Resource exhau